In [229]:
# Griffin Davis, The University of Texas at Dallas
# (C) 2022
# Data source:
# Chetty, Raj; Friedman, John; Hendren, Nathaniel; Jones, Maggie R.; Porter, Sonya R., 2022, 
# "Replication Data for: The Opportunity Atlas: Mapping the Childhood Roots of Social Mobility", 
# https://doi.org/10.7910/DVN/NKCQM1, Harvard Dataverse, V1, UNF:6:wwWmCZy1LUqtq02qHdCKFQ== [fileUNF] 

import os
import time
import logging
from importlib import reload
import pandas as pd
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from tqdm.notebook import tqdm

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, KFold

import torch
from torch import nn
from torch.distributions import Normal
nnF = nn.functional

if not os.path.exists('logs'):
    os.makedirs('logs')

reload(logging) # Notebook workaround
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(threadName)s] [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("logs/mdn.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Download data
!wget -nc https://personal.utdallas.edu/~gcd/data/tract_merged.csv
ds = pd.read_csv('tract_merged.csv')

File ‘tract_merged.csv’ already there; not retrieving.



In [188]:
# Get subset of columns
cols = ['id', 'hhinc_mean2000', 'mean_commutetime2000', 'frac_coll_plus2000', 'frac_coll_plus2010', 
        'med_hhinc1990', 'med_hhinc2016', 'popdensity2000', 'poor_share2010', 'poor_share2000', 
        'poor_share1990', 'gsmn_math_g3_2013', 'traveltime15_2010', 'emp2000', 'singleparent_share1990',
        'singleparent_share2010', 'singleparent_share2000', 
        'mail_return_rate2010', 'jobs_total_5mi_2015', 'jobs_highpay_5mi_2015', 
        'popdensity2010', 'job_density_2013', 'kfr_pooled_pooled_p1', 
        'kfr_pooled_pooled_p25', 'kfr_pooled_pooled_p50', 'kfr_pooled_pooled_p75', 'kfr_pooled_pooled_p100']

excluded = ['rent_twobed2015', 'ln_wage_growth_hs_grad', 'ann_avg_job_growth_2004_2013']

full_cols = cols + excluded

# Handle null data
ds_full = ds[ds.columns[ds.columns.isin(full_cols)]]
ds = ds[ds.columns[ds.columns.isin(cols)]]
ds = ds.dropna()

# Simple NN (Missing Values Removed)

In [189]:
# Shuffle split the data into training and test sets (75% / 25%)
train, test = train_test_split(ds)

train_X = train.loc[:,'hhinc_mean2000':'job_density_2013']
test_X = test.loc[:,'hhinc_mean2000':'job_density_2013']

percentiles = ['kfr_pooled_pooled_p1', 'kfr_pooled_pooled_p25', 'kfr_pooled_pooled_p50', 'kfr_pooled_pooled_p75']
train_Y = train.loc[:, percentiles[0]]
test_Y = test.loc[:, percentiles[0]]

# Reset indexes and convert Y to pd.Series
train_X.reset_index(drop=True, inplace=True)
train_Y = train_Y.reset_index(drop=True).squeeze()
test_X.reset_index(drop=True, inplace=True)
test_Y = test_Y.reset_index(drop=True).squeeze()

In [190]:
# Simple Mixture Density Network, without handling for missing input features
features = train_X.shape[1]

class Net(nn.Module):
    def __init__(self, features, hidden_dim, out_dim):
        super(Net, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(features, hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim, out_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        params = self.seq(x)
        mu, sigma = torch.tensor_split(params, params.shape[0], dim=0)
        
        return mu, sigma+1
    
    def loss(self, x, y):
        mu, sigma = self.forward(x)
        dist = Normal(mu, sigma)
        return -dist.log_prob(y)

In [199]:
def train_mdn(mdn, X, Y, optimizer, verbose):
    mdn.train()
    
    X.reset_index(drop=True, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    
    for index, row in X.iterrows():
        x = torch.tensor(np.double(row.values))
        y = torch.tensor(np.double(Y.iloc[index]))
        
        loss = mdn.loss(x, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if verbose and index % 10000 == 0:
            logging.info.info(f"index: {index} loss: {loss}")

In [192]:
def test_mdn(mdn, X, Y, verbose):
    mdn.eval()

    X.reset_index(drop=True, inplace=True)
    Y.reset_index(drop=True, inplace=True)

    test_loss = 0
    sq_er = []
    with torch.no_grad():
        for index, row in X.iterrows():
            x = torch.tensor(np.double(row.values))
            y = torch.tensor(np.double(Y.iloc[index]))
            loss = mdn.loss(x, y)
            test_loss += loss.item()

            mu, sigma = mdn.forward(x)

            sq_er.append((mu.item() - y)**2)

    test_loss /= test_X.shape[0]
    
    if verbose:
        logging.info(f"Avg test loss: {test_loss}")
        logging.info(f"Mean squared error: {np.mean(sq_er)}")
    
    return test_loss

In [194]:
# Determine number of Sigmoid activations to use in hidden layer
# 10-fold hyperparameter cross validation using training data
kf = KFold(n_splits=10)

losses = {}

# Setup folder to save hypervalidation models
hyperPath = 'hypervalidation'
if not os.path.exists(hyperPath):
    os.makedirs(hyperPath)

# Try each option for number of Sigmoid activations
for hidden_dim in tqdm(range(3, 30)):
    # Create network with that hyperparameter
    mdn = Net(features, hidden_dim, 2).double()
    optimizer = torch.optim.Adam(mdn.parameters(), lr=0.0001)

    # Do 10-fold cross validation and store results of tests in array
    dim_loss = []
    for train_index, test_index in kf.split(train_X):
        X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
        Y_train, Y_test = train_Y.iloc[train_index], train_Y.iloc[test_index]

        train_mdn(mdn, X_train, Y_train, optimizer, False)
        dim_loss.append(test_mdn(mdn, X_test, Y_test, False))
    
    # Store average test results for this hyperparameter option
    losses[hidden_dim] = np.mean(dim_loss)

    # Save the hypervalidation model
    torch.save(mdn, f'{hyperPath}/{hidden_dim}_activ_mdn.pt')

    logging.info("Dimension = " + str(hidden_dim) + ", Loss = " + str(losses[hidden_dim]))

# Select best hyperparameter
min_loss = np.inf
min_dim = 0
for hidden_dim in losses:
    if losses[hidden_dim] < min_loss:
        min_loss = losses[hidden_dim]
        min_dim = hidden_dim

logging.info("Selected number of sigmoid activations: " + str(min_dim))

  0%|          | 0/27 [00:00<?, ?it/s]

Dimension = 3, Loss = 0.2768381263867698
Dimension = 4, Loss = 0.2789649501573936
Dimension = 5, Loss = 0.29557487171902286
Dimension = 6, Loss = 0.29557487171902286
Dimension = 7, Loss = 0.27678297429819837
Dimension = 8, Loss = 0.27677964461746984
Dimension = 9, Loss = 0.2955755801044583
Dimension = 10, Loss = 0.2767739615945039
Dimension = 11, Loss = 0.2767723332257522
Dimension = 12, Loss = 0.27678039662463905
Dimension = 13, Loss = 0.27676444063356503
Dimension = 14, Loss = 0.29557487171902286
Dimension = 15, Loss = 0.2767818423319893
Dimension = 16, Loss = 0.27676558481836744
Dimension = 17, Loss = 0.2767831932562505
Dimension = 18, Loss = 0.2767636130155072
Dimension = 19, Loss = 0.2767720795720673
Dimension = 20, Loss = 0.29557487171902286
Dimension = 21, Loss = 0.27677251045612
Dimension = 22, Loss = 0.2767821537030486
Dimension = 23, Loss = 0.29557487171902286
Dimension = 24, Loss = 0.29557487171902286
Dimension = 25, Loss = 0.27674490638982185
Dimension = 26, Loss = 0.276776

In [195]:
# Retrain with full training dataset (selected 25)
mdn = Net(features, min_dim, 2).double()
optimizer = torch.optim.Adam(mdn.parameters(), lr=0.0001)

train_mdn(mdn, train_X, train_Y, optimizer, True)

# Save the final model
torch.save(mdn, f'simple_mdn.pt')

index: 0 loss: tensor([1.2939], dtype=torch.float64, grad_fn=<NegBackward0>)
index: 10000 loss: tensor([0.9458], dtype=torch.float64, grad_fn=<NegBackward0>)
index: 20000 loss: tensor([0.9245], dtype=torch.float64, grad_fn=<NegBackward0>)
index: 30000 loss: tensor([0.9222], dtype=torch.float64, grad_fn=<NegBackward0>)
index: 40000 loss: tensor([0.9221], dtype=torch.float64, grad_fn=<NegBackward0>)
index: 50000 loss: tensor([0.9198], dtype=torch.float64, grad_fn=<NegBackward0>)


In [ ]:
# Test with reserved data
test_mdn(mdn, test_X, test_Y, True)

Avg test loss: 0.9225062879034627
Mean squared error: 0.007135509397580172


0.9225062879034627

# NN with Expectation Maximization to Handle Missing Values

In [204]:
# Model for full dataset (including missing data)

# Shuffle split the data into training and test sets (75% / 25%)
train, test = train_test_split(ds_full)

train_X = train.loc[:,'hhinc_mean2000':'job_density_2013']
test_X = test.loc[:,'hhinc_mean2000':'job_density_2013']

percentiles = ['kfr_pooled_pooled_p1', 'kfr_pooled_pooled_p25', 'kfr_pooled_pooled_p50', 'kfr_pooled_pooled_p75']
train_Y = train.loc[:, percentiles[0]]
test_Y = test.loc[:, percentiles[0]]

# Reset indexes and convert Y to pd.Series
train_X.reset_index(drop=True, inplace=True)
train_Y = train_Y.reset_index(drop=True).squeeze()
test_X.reset_index(drop=True, inplace=True)
test_Y = test_Y.reset_index(drop=True).squeeze()

features = train_X.shape[1]
data_points = train_X.shape[0]

# Get missing indexes
train_Xmissing = np.isnan(train_X)
train_imissing = []
for m in range(data_points):
    row = []
    for i in range(features):
        if train_Xmissing.iloc[m][i]:
            row.append(i)
    train_imissing.append(row)

In [211]:
# Mixture Density Network with expectation maximization algorithm to handle missing data

# Store feature distribution parameters
feat_params = []

LOSS_SAMPLE_SIZE = 25
loss_generator = np.random.default_rng()

def init_feat_params(X):
    # Initialize feature parameters
    for n in range(features):
        mu = np.nanmean(X.iloc[:, n])
        sigma = np.nanstd(X.iloc[:, n])
        feat_params.append((mu, sigma))

def p_x(i, xi):
    mu, sigma = feat_params[i]
    dist = Normal(mu, sigma)
    p_xi  = torch.exp(dist.log_prob(torch.tensor(np.double(xi)))).item()
    return p_xi

class EMNet(nn.Module):
    def __init__(self, features, hidden_dim, out_dim):
        super(EMNet, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(features, hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim, out_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        params = self.seq(x)
        mu, sigma = torch.tensor_split(params, params.shape[0], dim=0)
        
        return mu, sigma+1

    # Evaluate q density for EM algorithm
    # For a given data point (x, y)
    def q(self, x, y, xmis, imis, mu, sigma):
        # p(y | x) with current mu, sigma
        y_dist = Normal(mu, sigma)

        # =======================
        # Numerator of q function
        # =======================

        # p(y | x)
        num = torch.exp(y_dist.log_prob(torch.tensor(np.double(y))))[0].item()

        # Product[ p(x_mis) ]
        st = time.time()
        for k in range(len(xmis)):
            num *= p_x(imis[k], xmis[k])

        # =========================
        # Denominator of q function
        # =========================

        # Function for the integral
        def int_func(*xmis_hat):
            # Get product of all p(xmis)
            prod = 1
            for i in range(len(xmis_hat)):
                prod *= p_x(imis[i], xmis_hat[i])
            x_hat = x
            f_i = 0
            # Replace missing values with current guesses
            for f in range(len(x_hat)):
                if np.isnan(x_hat[f]):
                    x_hat[f] = xmis_hat[f_i]
                    f_i+=1
            # Run X with current guesses through NN
            mu_hat, sigma_hat = self.forward(torch.tensor(np.double(x_hat)))
            dist = Normal(mu_hat, sigma_hat)
            # Return p(y|x)
            return torch.exp(dist.log_prob(torch.tensor(np.double(y))))[0].item()

        # Setup list of ranges corresponding to num of missing values
        ranges = [(-np.inf, np.inf) for i in range(len(imis))]

        # Integrate over each xmis, -inf to inf
        # st = time.time()
        den, err = integrate.nquad(int_func, ranges, opts={'epsabs': np.inf, 'epsrel': np.inf, 'limit': 10})
        # print(f"integral: {round(time.time()-st,1)}s")

        return num/den

    # Produce gradients of feature distribution parameters
    def grad_mu_xi(self, X, Y, mu_xi, sigma_xi, xi, imis):
        m = 0
        def int_func(*xmis_hat):
            nn_mu, nn_sigma = self.forward(torch.tensor(np.double(X[m])))
            
            qm = self.q(X[m], Y[m], xmis_hat, imis[m], nn_mu, nn_sigma)
            return qm * ((xi[m] / (sigma_xi**2)) - (mu_xi / (sigma_xi**2)))

        sum_ints = 0

        for m in range(len(X)):
            # Setup list of ranges corresponding to num of missing values
            ranges = [(-np.inf, np.inf) for i in range(len(imis[m]))]

            res, err = integrate.nquad(int_func, ranges, opts={'epsabs': np.inf, 'epsrel': np.inf, 'limit': 20})
            sum_ints += res

        return sum_ints

    def grad_sigma_xi(self, X, Y, mu_xi, sigma_xi, xi, imis):
        m = 0
        def int_func(*xmis_hat):
            nn_mu, nn_sigma = self.forward(torch.tensor(np.double(X[m])))

            qm = self.q(X[m], Y[m], xmis_hat, imis[m], nn_mu, nn_sigma)
            return qm * ((-1 / sigma_xi) - ((xi[m] - mu_xi) / (sigma_xi**3)))

        sum_ints = 0

        for m in range(len(X)):
            # Setup list of ranges corresponding to num of missing values
            ranges = [(-np.inf, np.inf) for i in range(len(imis[m]))]

            res, err = integrate.nquad(int_func, ranges, opts={'epsabs': np.inf, 'epsrel': np.inf, 'limit': 25})
            sum_ints += res

        return sum_ints
    
    def loss(self, X, Y, imis):
        logging.info("Calculating loss...")
        m = 0
        def int_func(*xmis_hat):
            # Replace NaNs with xmis_hat
            f_i = 0
            # Replace missing values with current guesses
            for f in range(len(X[m])):
                if np.isnan(X[m, f]):
                    X[m, f] = xmis_hat[f_i]
                    f_i+=1
            # Run X with current guesses through NN
            mu, sigma = self.forward(torch.tensor(np.double(X[m])))
            dist = Normal(mu, sigma)
            # Get p(y|x)
            log_p_y_x = dist.log_prob(torch.tensor(np.double(Y[m])))[0].item()

            # Get q_m(xmis_hat)
            qm = self.q(X[m], Y[m], xmis_hat, imis[m], mu, sigma)
            return qm * log_p_y_x

        sum_ints = 0

        # Randomly sample rows from the training data for each iteration
        X_sample = loss_generator.integers(0, high=X.shape[0], size=LOSS_SAMPLE_SIZE)

        for m in X_sample:
            # Handle case of no missing values
            if len(imis[m]) == 0:
                logging.info(f"row {m} : no missing values")
                mu, sigma = self.forward(torch.tensor(np.double(X[m])))
                dist = Normal(mu, sigma)
                # Get p(y|x)
                log_p_y_x = dist.log_prob(torch.tensor(np.double(Y[m])))[0].item()

                sum_ints += log_p_y_x
            else:
                # Setup list of ranges corresponding to num of missing values
                ranges = [(-np.inf, np.inf) for i in range(len(imis[m]))]
                start_time = time.time()
                res, err = integrate.nquad(int_func, ranges, opts={'epsabs': np.inf, 'epsrel': np.inf, 'limit': 20})
                logging.info(f"row {m} : completed {len(imis[m])} dim integral in {int(round(time.time()-start_time, 0))}s")
                sum_ints += res

        return -sum_ints

    def predict(self, x, imis):
        # def int_func(xmis_hat):
        #     # Replace NaNs with xmis_hat
        #     f_i = 0
        #     # Replace missing values with current guesses
        #     for f in range(len(x)):
        #         if np.isnan(x[f]):
        #             x[f] = xmis_hat[f_i]
        #             f_i+=1
        #     # Run X with current guesses through NN
        #     mu, sigma = self.forward(torch.tensor(np.double(x)))
        #     dist = Normal(mu, sigma)

        #     return qm

        # # Setup list of ranges corresponding to num of missing values
        # ranges = [(-np.inf, np.inf) for i in range(len(imis))]
        # integrate.nquad(int_func, ranges)
        pass

In [213]:
# Training constants
TRAIN_ITERATIONS = 100
FEAT_PARAM_STEP = 0.001

# Train neural network
def train_mdn(mdn, X, Y, optimizer, verbose):
    mdn.train()

    init_feat_params(X)

    X = X.to_numpy()
    Y = Y.to_numpy()
    
    for index in range(TRAIN_ITERATIONS):
        start_time = time.time()
        # Calculate NN loss        
        loss = mdn.loss(X, Y, train_imissing)
        
        # Backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update feature parameters using gradient descent
        for f in range(features):
            cur_mu, cur_sigma = feat_params[f]
            xi = X[:,f]
            grad_mu = mdn.grad_mu_xi(X, Y, cur_mu, cur_sigma, xi, train_imissing)
            grad_sigma = mdn.grad_sigma_xi(X, Y, cur_mu, cur_sigma, xi, train_imissing)

            feat_params[f] = (cur_mu+grad_mu, cur_sigma+grad_sigma)

        if verbose:
            logging.info(f"iteration: {index} nn loss: {loss} time elapsed: {int(round((time.time()-start_time)/60, 0))}min")

In [212]:
# Evaluate neural network
def test_mdn(mdn, X, Y, verbose):
    mdn.eval()

    X = X.to_numpy()
    Y = Y.to_numpy()

    # Get missing indexes
    Xmissing = np.isnan(X)
    imissing = []
    for m in range(X.shape[0]):
        row = []
        for i in range(features):
            if Xmissing[m, i]:
                row.append(i)
        imissing.append(row)
    
    sq_er = []
    with torch.no_grad():
        # Get total loss
        test_loss = mdn.loss(X, Y, imissing)

        # Get squared error for predictions
        # for index in range(X.shape[0]):
        #     row = X[index, :]
        #     x = torch.tensor(np.double(row))
        #     y = torch.tensor(np.double(Y[index]))

        #     mu, sigma = mdn.forward(x)

        #     sq_er.append((mu.item() - y)**2)

    test_loss /= test_X.shape[0]
    
    if verbose:
        logging.info(f"Avg test loss: {test_loss}")
        # print(f"Mean squared error: {np.mean(sq_er)}")
    
    return test_loss

In [235]:
%%capture output
# Retrain with full training dataset (selected 10)
mdn = EMNet(features, 10, 2).double()
optimizer = torch.optim.Adam(mdn.parameters(), lr=0.0001)

train_mdn(mdn, train_X, train_Y, optimizer, True)

# Save the final model
torch.save(mdn, f'em_mdn.pt')

2022-12-21 11:54:51,996 [MainThread] [INFO] Calculating loss...
2022-12-21 11:54:51,997 [MainThread] [INFO] row 12922 : no missing values
2022-12-21 11:54:52,001 [MainThread] [INFO] row 21060 : no missing values
2022-12-21 11:54:52,004 [MainThread] [INFO] row 1376 : no missing values


KeyboardInterrupt: 

In [234]:
output.show()